# CSU-MS2 cross-modal retireval 
### importing required libraries

In [2]:
import bisect
from infer import ModelInference
import numpy as np
import torch
from tqdm import tqdm
from matchms.importing import load_from_mgf
from matchms.Fragments import Fragments
import pandas as pd
import IsoSpecPy
import json
import os
import requests
import pubchempy as pc
from bs4 import BeautifulSoup
import matchms.filtering as msfilters

### define spectrum processing function and reference dataset searching function

In [3]:
def spectrum_processing(s):
    """This is how one would typically design a desired pre- and post-
    processing pipeline."""
    s = msfilters.normalize_intensities(s)
    s = msfilters.select_by_mz(s, mz_from=0, mz_to=1500)
    return s

def search_formula(formulaDB,mass, ppm): 
    mmin = mass - mass*ppm/10**6 
    mmax = mass + mass*ppm/10**6 
    lf = bisect.bisect_left(formulaDB['Exact mass'], mmin) 
    rg = bisect.bisect_right(formulaDB['Exact mass'], mmax) 
    formulas = list(formulaDB['Formula'][lf:rg]) 
    return formulas 

def search_structure(structureDB,formulas): 
    structures=pd.DataFrame()
    for formula in formulas:
        structure = structureDB[structureDB['MolecularFormula']==formula] 
        structures=pd.concat([structures,structure])
    return structures 

def search_structure_from_mass(structureDB,mass, ppm): 
    structures=pd.DataFrame()
    mmin = mass - mass*ppm/10**6 
    mmax = mass + mass*ppm/10**6 
    structures = structureDB[(structureDB['MonoisotopicMass'] >= mmin) & (structureDB['MonoisotopicMass'] <= mmax)]
    return structures 

### define structural feature vector calculation function

In [5]:
def get_feature(lst,save_name,model_inference,
                n=256,flag_get_value=False):

    print("Size of the library: ", len(lst))
    fn = model_inference.smiles_encode
    contexts = []


    print("start load batch")
    for i in range(0, len(lst), n):
       contexts.append(lst[i:i + n])

    print("start encode batch")
    result = [fn(i).cpu() for i in tqdm(contexts)]
    result = torch.cat(result, 0)
    if flag_get_value is True:
        if save_name is not None:
            torch.save((result, lst), save_name)
        return result, lst

def get_topK_result(library,ms_feature, smiles_feature, topK):
    indices = []
    scores = []
    candidates = []
    if topK >= len(library):
        topK = len(library)
    with torch.no_grad():
        ms_smiles_distances_tmp = (
            ms_feature @ smiles_feature.t()).cpu()
        scores_, indices_ = ms_smiles_distances_tmp.topk(topK,
                                                      dim=1,
                                                      largest=True,
                                                      sorted=True)
        candidates_=[library[i] for i in indices_.tolist()[0]]
        indices.append(indices_.tolist()[0])
        scores.append(scores_.tolist()[0])
        candidates.append(candidates_)
    return indices, scores, candidates

### loading model

In [6]:
config_path_low = "model/low_energy/checkpoints/config.yaml"
pretrain_model_path_low = "model/low_energy/checkpoints/model.pth"
model_inference_low = ModelInference(config_path=config_path_low,
                            pretrain_model_path=pretrain_model_path_low,
                            device="cpu")
config_path_median = "model/median_energy/checkpoints/config.yaml"
pretrain_model_path_median = "model/median_energy/checkpoints/model.pth"
model_inference_median = ModelInference(config_path=config_path_median,
                            pretrain_model_path=pretrain_model_path_median,
                            device="cpu")
config_path_high = "model/high_energy/checkpoints/config.yaml"
pretrain_model_path_high = "model/high_energy/checkpoints/model.pth"
model_inference_high = ModelInference(config_path=config_path_high,
                            pretrain_model_path=pretrain_model_path_high,
                            device="cpu") 

### create result save file

In [8]:
output_file='results/'
os.mkdir(output_file)

### upload spectra file, reference dataset file and collision energy file

In [9]:
ms_list=list(load_from_mgf("low_energy_spectra.mgf"))
reference_library = pd.read_csv('reference_dataset.csv')
collision_energy_file=pd.read_csv('collision_energy.csv')
collision_energy_lst = list(collision_energy_file['CollisionEnergy'])

 ### perform cross-modal retrieval for a list of spectra

In [10]:
for i in tqdm(range(len(ms_list))):
            result=pd.DataFrame()
            spectrum = ms_list[i]
            spectrum = spectrum_processing(spectrum)
            ms_feature_low = model_inference_low.ms2_encode([spectrum])
            ms_feature_median = model_inference_median.ms2_encode([spectrum])
            ms_feature_high = model_inference_high.ms2_encode([spectrum])
            
            # The precursor mass and collison energy can also be set manually 
            query_ms = float(spectrum.metadata['precursor_mz'])-1.008
            #collision_energy = int(spectrum.metadata['collision_energy'])
            collision_energy = collision_energy_lst[i]
            search_res=search_structure_from_mass(reference_library, query_ms,30)
            smiles_lst = list(search_res['SMILES'])

            smiles_feature_low, smiles_lst1 = get_feature(smiles_lst,save_name=None,
                model_inference=model_inference_low,n=1,flag_get_value=True)
            smiles_feature_median, smiles_lst2 = get_feature(smiles_lst,save_name=None,
                model_inference=model_inference_median,n=1,flag_get_value=True)
            smiles_feature_high, smiles_lst3 = get_feature(smiles_lst,save_name=None,
                model_inference=model_inference_high,n=1,flag_get_value=True)
            
            low_similarity = ms_feature_low @ smiles_feature_low.t()
            median_similarity = ms_feature_median @ smiles_feature_median.t()
            high_similarity = ms_feature_high @ smiles_feature_high.t()
            low_similarity = low_similarity.numpy()
            median_similarity = median_similarity.numpy()
            high_similarity = high_similarity.numpy()
            
            weight1 = (1/abs(collision_energy-10+1e-10))/((1/abs(collision_energy-10+1e-10))+(1/abs(collision_energy-20+1e-10))+(1/abs(collision_energy-40+1e-10)))
            weight2 = (1/abs(collision_energy-20+1e-10))/((1/abs(collision_energy-10+1e-10))+(1/abs(collision_energy-20+1e-10))+(1/abs(collision_energy-40+1e-10)))
            weight3 = (1/abs(collision_energy-40+1e-10))/((1/abs(collision_energy-10+1e-10))+(1/abs(collision_energy-20+1e-10))+(1/abs(collision_energy-40+1e-10)))
   
            weighted_similarity = weight1 * low_similarity + weight2 * median_similarity + weight3 * high_similarity
            weighted_similarity = np.squeeze(weighted_similarity, axis=0)
            weighted_similarity_scores=[(smiles_lst1[i],weighted_similarity[i]) for i in range(len(smiles_lst1))]
            weighted_similarity_scores.sort(key=lambda x: x[1], reverse=True)
            results = pd.DataFrame({'SMILES':[x[0] for x in weighted_similarity_scores],'Score':[x[1] for x in weighted_similarity_scores],'Rank':list(range(1,len(smiles_lst1)+1))})
            results.to_csv(output_file+'spectrum'+str(i)+'.csv')

  0%|                                                                                          | 0/263 [00:00<?, ?it/s]

Size of the library:  143
start load batch
start encode batch



100%|████████████████████████████████████████████████████████████████████████████████| 143/143 [00:02<00:00, 68.29it/s]


Size of the library:  143
start load batch
start encode batch



100%|████████████████████████████████████████████████████████████████████████████████| 143/143 [00:01<00:00, 75.30it/s]


...


100%|████████████████████████████████████████████████████████████████████████████████| 305/305 [00:04<00:00, 73.28it/s]


Size of the library:  305
start load batch
start encode batch



100%|████████████████████████████████████████████████████████████████████████████████| 305/305 [00:03<00:00, 86.22it/s]


Size of the library:  305
start load batch
start encode batch



100%|████████████████████████████████████████████████████████████████████████████████| 263/263 [23:34<00:00,  5.38s/it]


In [19]:
print(results.head())

                                              SMILES     Score  Rank
0  CSC1=[N+](C2=CC=CC=C2)C2C3=CC=CC=C3N=C(C3=CC=C...  0.151581     1
1  C#CC(P)=NC(=[NH2+])C1=CC=C(C2=CC=CC3=CC=C(C(=C...  0.148580     2
2  CC1=C(C(=O)N2CCC(N3C=NC=N3)CC2)N=NN1C1=CC=CC(C...  0.126529     3
3  CC(C)(C)C1=CC=C2C(C3=CC=CC=C3C(=O)[O-])=C3C=CC...  0.119582     4
4   CC1=NC(COC2=CC=CC=C2C=CC(=O)NC2CC(=O)N(C)C2)=CS1  0.117753     5
